In [3]:
import pandas as pd


In [4]:
sales = pd.read_excel("sales_data.xlsx")
customers = pd.read_excel("customers.xlsx")
products = pd.read_excel("products.xlsx")
marketing = pd.read_excel("marketing_spend.xlsx")


In [5]:
# Check info
print(sales.info())
print(sales.isnull().sum())

# Drop rows with missing values (if any)
sales.dropna(inplace=True)

# Ensure data types
sales['sale_date'] = pd.to_datetime(sales['sale_date'])
sales['quantity'] = sales['quantity'].astype(int)
sales['unit_price'] = sales['unit_price'].astype(float)

# Recalculate total_price
sales['total_price'] = (sales['quantity'] * sales['unit_price']).round(2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_id      1000 non-null   int64         
 1   customer_id  1000 non-null   int64         
 2   product_id   1000 non-null   int64         
 3   quantity     1000 non-null   int64         
 4   sale_date    1000 non-null   datetime64[ns]
 5   unit_price   1000 non-null   float64       
 6   total_price  1000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 54.8 KB
None
sale_id        0
customer_id    0
product_id     0
quantity       0
sale_date      0
unit_price     0
total_price    0
dtype: int64


In [6]:

print(customers.info())
customers['join_date'] = pd.to_datetime(customers['join_date'])
customers.drop_duplicates(subset='customer_id', inplace=True)

# Optional: standardize regions
customers['region'] = customers['region'].str.title().str.strip()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   customer_id    100 non-null    int64         
 1   customer_name  100 non-null    object        
 2   region         100 non-null    object        
 3   join_date      100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.3+ KB
None


In [7]:
products.drop_duplicates(subset='product_id', inplace=True)
products['product_name'] = products['product_name'].str.strip()
products['category'] = products['category'].str.title().str.strip()


In [8]:
marketing['month'] = pd.to_datetime(marketing['month'], format="%Y-%m")
marketing['budget'] = marketing['budget'].astype(int)



In [9]:
merged_sales = sales.merge(customers, on="customer_id", how="left")\
                    .merge(products, on="product_id", how="left")

# Check if merge introduced nulls
print(merged_sales.isnull().sum())

# Optional: drop rows with missing merge info
merged_sales.dropna(inplace=True)


sale_id          0
customer_id      0
product_id       0
quantity         0
sale_date        0
unit_price       0
total_price      0
customer_name    0
region           0
join_date        0
product_name     0
category         0
dtype: int64


In [10]:
# Export to CSV or load to SQL
merged_sales.to_csv("cleaned_sales.csv", index=False)
customers.to_csv("cleaned_customers.csv", index=False)
products.to_csv("cleaned_products.csv", index=False)
marketing.to_csv("cleaned_marketing.csv", index=False)


In [12]:
import pandas as pd
from sqlalchemy import create_engine

# Replace with your actual MySQL credentials
user = "root"
password = "root"
host = "localhost:3307"
database = "retail_db"

# Create SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# Load cleaned CSVs
sales = pd.read_csv("cleaned_sales.csv")
customers = pd.read_csv("cleaned_customers.csv")
products = pd.read_csv("cleaned_products.csv")
marketing = pd.read_csv("cleaned_marketing.csv")

# Upload to MySQL
customers.to_sql("customers", engine, if_exists="replace", index=False)
products.to_sql("products", engine, if_exists="replace", index=False)
marketing.to_sql("marketing_spend", engine, if_exists="replace", index=False)
sales.to_sql("sales_data", engine, if_exists="replace", index=False)


1000